<a href="https://colab.research.google.com/github/Harshkotkar/AI-agents-/blob/main/Webbase_loader_(Rag).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U langchain langchain_community langchain_core
!pip install langchain-text-splitter
!pip install faiss-cpu
!pip install pypdf
!pip install bs4
!pip install langchain_google_genai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.8/93.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 108.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.2/471.2 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.3/208.3 kB 22.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain_core
    Found

In [ ]:
!pip install langchain_chroma

In [ ]:
import getpass
import os
os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API key: ")




Enter your Google API key: ··········


In [ ]:
# documetn datastructure what exactly is it ?
"""The Document data structure in Langchain is a container for text data and its associated metadata.
 It has two main parts: page_content which holds the text itself, and metadata which is a dictionary for storing extra information like the source or page number.
  This helps Langchain organize and understand the text it's working with."""



"The Document data structure in Langchain is a container for text data and its associated metadata.\n It has two main parts: page_content which holds the text itself, and metadata which is a dictionary for storing extra information like the source or page number.\n  This helps Langchain organize and understand the text it's working with."

In [ ]:
import bs4
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

In [ ]:
llm=ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp",temperature=0)


In [ ]:
loder=WebBaseLoader(web_path="https://www.hri.org.au/health/learn/your-body/the-human-heart",
                    bs_kwargs=dict(
                        parse_only=bs4.SoupStrainer(id="main-content")
                    ))

In [ ]:
docs=loder.load()

In [ ]:
text_split=RecursiveCharacterTextSplitter(chunk_size=1000,
                                          chunk_overlap=200)
split=text_split.split_documents(docs)

In [ ]:
split[:3]

[Document(metadata={'source': 'https://www.hri.org.au/health/learn/your-body/the-human-heart'}, page_content='What is the heart?\n                    \n\n\nThe heart is one of the most important organs in the human body. It is a powerful fist-sized muscle that pumps blood around the body through a network of blood vessels – together, the heart and blood vessels make up the body’s cardiovascular system. \nOur hearts beat on average 72 times every minute – over 100,000 times per day. Each minute, vital materials are circulated in our blood and waste products are removed. Each minute counts in helping our body function.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n            Download a fun poster to help children learn more about the heart (2.2MB PDF)\n        \n\n\n\n\n\n\n\n\n\n\n\n\n                        What side is the heart located?'),
 Document(metadata={'source': 'https://www.hri.org.au/health/learn/your-body/the-human-heart'}, page_content='Download a fun poster to help children l

In [ ]:
model_name="BAAI/bge-Base-en-v1.5"
model=HuggingFaceEmbeddings(model_name=model_name,
                                model_kwargs={"device":"cpu"},
                                encode_kwargs={"normalize_embeddings":True})

/tmp/ipython-input-1255604079.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  model=HuggingFaceEmbeddings(model_name=model_name,
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
db=Chroma.from_documents(documents=split,
                         embedding=model)

In [ ]:
retriver=db.as_retriever() # its work is to find the most relevant parts of your documents that relate to your question

In [ ]:
prompt = PromptTemplate.from_template("""Answer  the question based only on the provided context answer as you are answering a childern of "12" years:

<context>
{context}
</context>

Question: {input}""")

In [ ]:
def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
chain=({"context":retriver |format_docs ,"input": RunnablePassthrough()} # simply takes the input it receives and passes it along without changing it. It's useful for when you want to include the original input in a chain of operations
       | prompt
       | llm
       | StrOutputParser())

In [ ]:
query="What is the heart?"
result=chain.invoke(query)
print(result)

The heart is like a super important muscle in your chest, about the size of your fist! It's like a pump that sends blood all around your body through tiny tubes. It works all day and night, even when you're sleeping, to keep you alive and healthy!
